In [1]:
from __future__ import division

import math as math
import numpy as np
import scipy as scp
import pylab as pyl
from matplotlib import cm
import matplotlib.pyplot as plt
import scipy.io as sio
from matplotlib.pyplot import imshow as imageplot
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.image as mpimg
import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%load_ext autoreload
%autoreload 2


In [2]:
def grad_x(img, adjoint):
    sx , sy = np.shape(img)
    diff_x = np.copy(img)
    
    if adjoint==0: # ça c'est pour calculer le gradient de l'opérateur lui meme
        for x in range(sx):
            if x==sx-1:
                xnext=0
            else:
                xnext=x+1
            for y in range(sy):
                diff_x[x,y] = img[xnext,y]- img[x,y]
    else:  # là on calcule le gradien de l'operateur adjoint
        for x in range(sx):
            if x==0:
                xprev=sx-1
            else:
                xprev=x-1
            for y in range(sy):
                diff_x[x,y] = img[xprev,y]- img[x,y]
    
    return diff_x
    
def grad_y(img, adjoint):
    sx , sy = np.shape(img)
    diff_y =  np.copy(img)

    if adjoint==0:
       
        for y in range(sy):
            if y==sy-1:
                ynext=0
            else:
                ynext=y+1
            for x in range(sx):
                diff_y[x,y] = img[x,ynext]- img[x,y]
    else:
        for y in range(sy):
            if y==0:
                yprev=sy-1
            else:
                yprev=y-1
            for x in range(sx):
                diff_y[x,y] = img[x,yprev]- img[x,y]
    
    return diff_y

def grad(v):
    gr = [grad_x(v),grad_y(v)]
    return 

In [5]:
# on definit la fonction D term attaché au donnée

def M(t,v1):
    t1 = t[0,:,:]
    t2 = t[1,:,:]
    dx = grad_x(v1)
    dy = grad_y(v1)
    return dx*t1 + dy*t2
def M_adjoint(s,v):
    dx = grad_x(v,0)
    dy = grad_y(v,0)
    m,n = v.shape
    A = np.zeros(2,m,n)
    A[0,:,:] = dx * s
    A[1,:,:] = dy * s
    return A
def D(t,v1,v2):
    return np.linalg.norm(v1 - v2 + M(t,v1))

In [9]:
# on definit la fonction R term de régularisation
def phi(t1,t2,eps):
    return np.sqrt(t1+t2+eps)
def R(t,eps):
    t1 = t[0,:,:]
    t2 = t[1,:,:]
    dt1x = grad_x(t1,0)
    dt1y = grad_y(t1,0) 
    dt2x = grad_x(t2,0) 
    dt2y = grad_y(t2,0) 
    normt1 = dt1x**2 +dt1y**2
    normt2 = dt2x**2 +dt2y**2
    return sum(phi(normt1,normt2,eps))

In [11]:
#definir  l'energie E
def E(v1,t,v2,eps):
    return D(t,v1,v2) + R(t,eps)